# Notebook de Traduction Anglais-Français (Seq2Seq) avec Embeddings

Ce notebook implémente un modèle encodeur-décodeur pour la traduction de l'anglais vers le français.

**Objectif :** Remplacer l'approche "caractère par caractère" et "one-hot" par :
1.  Un **vocabulaire de mots** (limité à 2056 tokens).
2.  Des couches `Embedding` pour représenter les mots.

## 1. Téléchargement et Chargement des Données

Nous allons d'abord télécharger le fichier `fra.txt` qui contient les paires de phrases anglais-français.

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Embedding, LSTM, Dense, TextVectorization
from tensorflow.keras.models import Model
from tensorflow.keras.utils import get_file
import numpy as np
import re
import string

# Télécharger le fichier de données
data_path = get_file(
    "fra.txt",
    origin="http://storage.googleapis.com/download.tensorflow.org/data/fra-eng.zip",
    extract=True
)
# Le chemin est maintenant un répertoire, nous devons trouver le fichier .txt à l'intérieur
data_path = data_path.replace(".zip", "/fra.txt")

# Lire le fichier et afficher les premières lignes
with open(data_path+'/fra.txt', "r", encoding="utf-8") as f:
    lines = f.read().split("\n")

print("Exemples de lignes du fichier fra.txt:")
for line in lines[:5]:
    print(line)

print(f"\nNombre total de paires de phrases: {len(lines)}")

Exemples de lignes du fichier fra.txt:
Go.	Va !
Hi.	Salut !
Run!	Cours !
Run!	Courez !
Who?	Qui ?

Nombre total de paires de phrases: 167131


## 2. Création du Dataset (Paires Anglais-Français)

Nous allons parser le fichier et créer une liste de paires `(input_text, target_text)`.

Pour que le décodeur sache quand commencer et finir, nous ajoutons des tokens spéciaux :
* `[start]` au début des séquences cibles (français).
* `[end]` à la fin des séquences cibles.

Nous allons également limiter la taille du dataset pour un entraînement plus rapide.

In [62]:
num_samples = 120000  # On utilise 30 000 échantillons pour un entraînement rapide
input_texts = []
target_texts = []

for line in lines[: min(num_samples, len(lines) - 1)]:
    try:
        input_text, target_text= line.split("\t")
        # Ajout des tokens de début et de fin pour la cible (français)
        target_text = "[start] " + target_text + " [end]"
        input_texts.append(input_text)
        target_texts.append(target_text)
    except ValueError:
        # Ignorer les lignes mal formées
        pass

print("\nExemple de paire (Entrée, Cible) après traitement:")
print(f"Entrée (Anglais): {input_texts[10]}")
print(f"Cible (Français): {target_texts[10]}")
print(f"\nTaille du dataset utilisé: {len(input_texts)}")


Exemple de paire (Entrée, Cible) après traitement:
Entrée (Anglais): Stop!
Cible (Français): [start] Stop ! [end]

Taille du dataset utilisé: 120000


## 3, 4 & 5. Tokenization et Vectorisation (Remplacement des étapes 3, 4, 5)

Au lieu de trouver des *caractères* et de faire du *one-hot* (étapes 3, 4, 5 de votre plan), nous allons utiliser la couche `TextVectorization` de Keras. C'est la méthode moderne pour :
1.  Créer un vocabulaire de mots (étape 3).
2.  Transformer les chaînes de caractères en chaînes d'entiers (étape 4).
3.  Préparer les données pour une couche `Embedding` (remplace l'étape 5).

Nous fixons la taille du vocabulaire à **2056** comme demandé.

In [63]:
VOCAB_SIZE = 256
EMBEDDING_DIM = 24
LATENT_DIM = 256  # États internes de l'LSTM, comme demandé (étape 6a)
BATCH_SIZE = 32

# Fonction de standardisation pour nettoyer le texte
def custom_standardization(input_string):
    # Mettre en minuscule
    lowercase = tf.strings.lower(input_string)
    # Retirer les balises HTML (au cas où)
    stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
    # Gérer la ponctuation. [start] et [end] sont des mots, ne pas les séparer.
    # Garder les crochets pour nos tokens spéciaux
    punctuation_to_remove = '"#$%&()*+,-./:;<=>@\\^_`{|}~'
    # Retirer la ponctuation
    no_punctuation = tf.strings.regex_replace(
        stripped_html, f'[{re.escape(punctuation_to_remove)}]', ''
    )
    # Séparer les points d'interrogation et d'exclamation
    no_punctuation = tf.strings.regex_replace(no_punctuation, "([?.!,])", r" \1 ")
    return no_punctuation

# Création de la couche de vectorisation pour l'Anglais (Entrée)
input_vectorization = TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode="int",
    standardize=custom_standardization
)

# Création de la couche de vectorisation pour le Français (Cible)
target_vectorization = TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode="int",
    standardize=custom_standardization
)

# Adapter (construire le vocabulaire) les couches sur nos textes
input_vectorization.adapt(input_texts)
target_vectorization.adapt(target_texts)

print("\n--- Vocabulaire Français (Cible) ---")
print(target_vectorization.get_vocabulary()[:20])

print("\n--- Exemple de vectorisation ---")
sample_input = input_texts[10]
sample_target = target_texts[10]
print(f"Texte Anglais: {sample_input}")
print(f"Vectorisé: {input_vectorization([sample_input]).numpy()}")
print(f"Texte Français: {sample_target}")
print(f"Vectorisé: {target_vectorization([sample_target]).numpy()}")


--- Vocabulaire Français (Cible) ---
['', '[UNK]', np.str_('[start]'), np.str_('[end]'), np.str_('je'), np.str_('?'), np.str_('de'), np.str_('pas'), np.str_('que'), np.str_('ne'), np.str_('vous'), np.str_('le'), np.str_('à'), np.str_('tom'), np.str_('il'), np.str_('la'), np.str_('est'), np.str_('un'), np.str_('nous'), np.str_('tu')]

--- Exemple de vectorisation ---
Texte Anglais: Stop!
Vectorisé: [[145  84]]
Texte Français: [start] Stop ! [end]
Vectorisé: [[ 2  1 26  3]]


### Création du `tf.data.Dataset`

Nous transformons nos listes en un `tf.data.Dataset` pour l'efficacité. C'est ici que nous préparons les paires pour le *teacher forcing* :

* `decoder_input`: `[start] Je suis...`
* `decoder_target`: `Je suis... [end]`

Le modèle apprendra à prédire le mot `Je` en voyant `[start]`, puis `suis` en voyant `Je`, etc.

In [64]:
def vectorize_pair(input_text, target_text):
    input_vec = input_vectorization([input_text])
    target_vec = target_vectorization([target_text])
    return input_vec[0], target_vec[0] # [0] pour retirer la dimension batch

def format_dataset(input_vec, target_vec):
    # Prépare les paires pour le teacher forcing
    # input_vec: [1, 2, 3]
    # target_vec: [10, 11, 12, 13] ([start], mot1, mot2, [end])

    # decoder_input: [10, 11, 12] ([start], mot1, mot2)
    decoder_input = target_vec[:-1]

    # decoder_target: [11, 12, 13] (mot1, mot2, [end])
    decoder_target = target_vec[1:]

    # Retourne: ({input_1: ..., input_2: ...}, output)
    return ({'encoder_input': input_vec, 'decoder_input': decoder_input}, decoder_target)

# Créer le dataset de base
dataset = tf.data.Dataset.from_tensor_slices((input_texts, target_texts))

# Vectoriser les textes
# Note: tf.py_function est nécessaire car TextVectorization n'est pas (encore) parfaitement
# compatible avec .map() si les formes ne sont pas statiques.
# Une alternative est de tout vectoriser en mémoire d'abord, mais c'est moins flexible.

# Pour simplifier et garantir la compatibilité, nous allons vectoriser en mémoire (style NumPy)
input_vecs = input_vectorization(input_texts)
target_vecs = target_vectorization(target_texts)

# Trouver la longueur maximale pour le padding
max_input_seq_len = input_vecs.shape[1]
max_target_seq_len = target_vecs.shape[1]

print(f"\nMax seq length (input): {max_input_seq_len}")
print(f"Max seq length (target): {max_target_seq_len}")

# Créer le dataset final à partir des tenseurs vectorisés
dataset = tf.data.Dataset.from_tensor_slices((input_vecs, target_vecs))
dataset = dataset.map(format_dataset, num_parallel_calls=tf.data.AUTOTUNE)
dataset = dataset.shuffle(buffer_size=2048).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# Vérifier un batch
for (inputs, target) in dataset.take(1):
    print("\n--- Forme d'un batch --- ")
    print(f"inputs['encoder_input'].shape: {inputs['encoder_input'].shape}")
    print(f"inputs['decoder_input'].shape: {inputs['decoder_input'].shape}")
    print(f"target.shape: {target.shape}")


Max seq length (input): 11
Max seq length (target): 18

--- Forme d'un batch --- 
inputs['encoder_input'].shape: (32, 11)
inputs['decoder_input'].shape: (32, 17)
target.shape: (32, 17)


## 6. Construction du Réseau de Neurones (Encodeur-Décodeur)

Nous construisons le modèle en utilisant l'API fonctionnelle de Keras.

* **Couche d'Entrée (Anglais)**: `encoder_input`
* **Couche d'Embedding (Anglais)**: Transforme les entiers en vecteurs denses.
* **a. Encodeur (LSTM)**: Couche LSTM avec 256 états internes (`LATENT_DIM`). On récupère ses états `h` et `c`.
* **Couche d'Entrée (Français)**: `decoder_input`
* **Couche d'Embedding (Français)**: Une *autre* couche d'embedding pour le français.
* **b. Décodeur (LSTM)**: Utilise les états `h` et `c` de l'encodeur comme `initial_state`.
* **c. Couche Dense de Sortie**: Prédit le mot suivant. La taille est `VOCAB_SIZE` (2056) avec une activation `softmax`.

In [65]:
import tensorflow as tf

# --- Définition de l'Encodeur ---
encoder_input = keras.Input(shape=(None,), name="encoder_input")

# Couche d'Embedding (remplace one-hot)
encoder_embedding_layer = Embedding(
    VOCAB_SIZE,
    EMBEDDING_DIM,
    embeddings_initializer='glorot_uniform',
    embeddings_regularizer=tf.keras.regularizers.L2(l2=0.01)
)
encoder_emb = encoder_embedding_layer(encoder_input)

# a. Couche LSTM de l'encodeur
encoder_lstm = LSTM(LATENT_DIM, return_state=True, name="encoder_lstm")
_, encoder_state_h, encoder_state_c = encoder_lstm(encoder_emb)

# Garder les états de l'encodeur
encoder_states = [encoder_state_h, encoder_state_c]

# --- Définition du Décodeur ---
decoder_input = keras.Input(shape=(None,), name="decoder_input")

# Couche d'Embedding (séparée pour le décodeur)
decoder_embedding_layer = Embedding(
    VOCAB_SIZE,
    EMBEDDING_DIM,
    embeddings_initializer='glorot_uniform',
    embeddings_regularizer=tf.keras.regularizers.L2(l2=0.01)
)
decoder_emb = decoder_embedding_layer(decoder_input)

# b. Couche LSTM du décodeur
# L'état initial est l'état de l'encodeur
decoder_lstm = LSTM(LATENT_DIM, return_sequences=True, return_state=True, name="decoder_lstm")
decoder_outputs, _, _ = decoder_lstm(decoder_emb, initial_state=encoder_states)

# c. Couche Dense de sortie
decoder_dense = Dense(VOCAB_SIZE, activation="softmax", name="decoder_dense")
decoder_output = decoder_dense(decoder_outputs)

# --- Création du Modèle (pour l'entraînement) ---
model = Model([encoder_input, decoder_input], decoder_output)

model.summary()

Model: "functional_18"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ encoder_input       │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_input       │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_12        │ (None, None, 24)  │      6,144 │ encoder_input[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_13        │ (None, None, 24)  │      6,144 │ decoder_input[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ encoder_lstm (LSTM) │ [(None, 256),     │    287,744 │ embedding_12[0][… │
│                     │ (None, 256),      │            │                   │
│                     │ (None, 256)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_lstm (LSTM) │ [(None, None,     │    287,744 │ embedding_13[0][… │
│                     │ 256), (None,      │            │ encoder_lstm[0][… │
│                     │ 256), (None,      │            │ encoder_lstm[0][… │
│                     │ 256)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_dense       │ (None, None, 256) │     65,792 │ decoder_lstm[0][… │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 653,568 (2.49 MB)

 Trainable params: 653,568 (2.49 MB)

 Non-trainable params: 0 (0.00 B)

## 8. Compilation et Entraînement

Nous compilons le modèle. Notez l'utilisation de `sparse_categorical_crossentropy` : c'est la fonction de perte correcte lorsque vos cibles sont des entiers (ID de mots) et que votre sortie est une distribution de probabilité (softmax), ce qui évite d'avoir à convertir les cibles en one-hot.

In [66]:
model.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

# L'entraînement peut prendre du temps (5-15 min sur un GPU Colab)
epochs = 15 # Increased from 20 to 50

history = model.fit(
    dataset,
    epochs=epochs,
    validation_data=dataset.take(int(0.1 * len(dataset))) # Utiliser une petite partie pour la validation
)

Epoch 1/15
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 38s 10ms/step - accuracy: 0.8033 - loss: 1.0329 - val_accuracy: 0.8317 - val_loss: 0.9065
Epoch 2/15
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 35s 9ms/step - accuracy: 0.8564 - loss: 0.6632 - val_accuracy: 0.8293 - val_loss: 0.8412
Epoch 3/15
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 41s 9ms/step - accuracy: 0.8667 - loss: 0.5966 - val_accuracy: 0.8377 - val_loss: 0.7385
Epoch 4/15
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 36s 10ms/step - accuracy: 0.8718 - loss: 0.5588 - val_accuracy: 0.8437 - val_loss: 0.6996
Epoch 5/15
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 36s 9ms/step - accuracy: 0.8762 - loss: 0.5308 - val_accuracy: 0.8528 - val_loss: 0.6517
Epoch 6/15
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 35s 9ms/step - accuracy: 0.8794 - loss: 0.5082 - val_accuracy: 0.8624 - val_loss: 0.6101
Epoch 7/15
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 42s 10ms/step - accuracy: 0.8824 - loss: 0.4903 - val_accuracy: 0.8588 - val_loss: 0.6376
Epoch 8/15
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 41s 10ms/step - accuracy: 0.8842 - los

KeyboardInterrupt: 

## 9. Création des Modèles d'Inférence

Pour la traduction (inférence), nous ne pouvons pas utiliser le *teacher forcing*. Nous devons prédire un mot à la fois et réinjecter cette prédiction dans le modèle pour générer le mot suivant.

Pour cela, nous divisons notre modèle entraîné en deux parties, comme vous l'avez décrit.

### 9.a. Modèle d'Encodeur

Prend la phrase en anglais et renvoie les états internes `(h, c)`.

In [67]:
encoder_model = Model(encoder_input, encoder_states)
encoder_model.summary()

Model: "functional_19"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ encoder_input (InputLayer)      │ (None, None)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_12 (Embedding)        │ (None, None, 24)       │         6,144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder_lstm (LSTM)             │ [(None, 256), (None,   │       287,744 │
│                                 │ 256), (None, 256)]     │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 293,888 (1.12 MB)

 Trainable params: 293,888 (1.12 MB)

 Non-trainable params: 0 (0.00 B)

### 9.b. Modèle de Décodeur

Prend le mot précédent (ou `[start]`) et les états internes `(h, c)`, et renvoie la prédiction pour le mot suivant ainsi que les nouveaux états `(h, c)`.

In [68]:
# Entrées pour les états du décodeur
decoder_state_input_h = keras.Input(shape=(LATENT_DIM,), name="decoder_state_h")
decoder_state_input_c = keras.Input(shape=(LATENT_DIM,), name="decoder_state_c")
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# Entrée pour le token (un seul mot à la fois)
decoder_input_single = keras.Input(shape=(1,), name="decoder_input_single")

# Réutiliser les couches entraînées
decoder_emb_inf = decoder_embedding_layer(decoder_input_single)
decoder_outputs_inf, decoder_state_h_inf, decoder_state_c_inf = decoder_lstm(
    decoder_emb_inf, initial_state=decoder_states_inputs
)
decoder_states_inf = [decoder_state_h_inf, decoder_state_c_inf]

decoder_output_inf = decoder_dense(decoder_outputs_inf)

# Création du modèle d'inférence
decoder_model = Model(
    [decoder_input_single] + decoder_states_inputs,
    [decoder_output_inf] + decoder_states_inf
)

decoder_model.summary()

Model: "functional_20"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ decoder_input_sing… │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_13        │ (None, 1, 24)     │      6,144 │ decoder_input_si… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_state_h     │ (None, 256)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_state_c     │ (None, 256)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_lstm (LSTM) │ [(None, 1, 256),  │    287,744 │ embedding_13[1][… │
│                     │ (None, 256),      │            │ decoder_state_h[… │
│                     │ (None, 256)]      │            │ decoder_state_c[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_dense       │ (None, 1, 256)    │     65,792 │ decoder_lstm[1][… │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 359,680 (1.37 MB)

 Trainable params: 359,680 (1.37 MB)

 Non-trainable params: 0 (0.00 B)

### 9.c. Fonction de Traduction (Inférence)

C'est ici que nous assemblons le tout :
1.  Passer la séquence anglaise dans `encoder_model` pour obtenir les états.
2.  Commencer une boucle avec le token `[start]`.
3.  Utiliser `decoder_model` pour prédire le mot suivant.
4.  Récupérer l'ID du mot (avec `argmax`).
5.  Si c'est `[end]`, arrêter. Sinon, ajouter le mot à la traduction.
6.  Réinjecter le nouvel ID et les nouveaux états dans le `decoder_model` et continuer.

In [71]:
import numpy as np
import tensorflow as tf

# Créer des dictionnaires pour convertir les ID en mots (et vice-versa)
input_vocab = input_vectorization.get_vocabulary()
target_vocab = target_vectorization.get_vocabulary()

input_word_to_id = {word: i for i, word in enumerate(input_vocab)}
target_id_to_word = {i: word for i, word in enumerate(target_vocab)}

# Obtenir les ID des tokens spéciaux
start_token_id = target_vectorization(["[start]"]).numpy()[0, 0]
end_token_id = target_vectorization(["[end]"]).numpy()[0, 0]

def translate(input_sentence, temperature=0.0):
    # 1. Vectoriser la phrase d'entrée
    input_vec = input_vectorization([input_sentence])

    # 2. Obtenir les états initiaux de l'encodeur
    states_value = encoder_model.predict(input_vec, verbose=0)

    # 3. Préparer le premier token pour le décodeur ([start])
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = start_token_id

    decoded_sentence = ""
    stop_condition = False
    i = 0
    print(f"\nTranslating: '{input_sentence}' (temperature={temperature})")
    # Limite pour éviter les boucles infinies
    while not stop_condition:
        # 4. Prédire le mot suivant (output_tokens sont les probabilités softmax)
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value, verbose=0)

        # Obtenir la distribution de probabilité pour le mot suivant
        preds = output_tokens[0, -1, :]

        # Appliquer la température
        if temperature > 0:
            preds = np.log(preds + 1e-10) / temperature # Ajouter un epsilon pour éviter log(0)
            exp_preds = np.exp(preds)
            preds = exp_preds / np.sum(exp_preds) # Re-normaliser
            sampled_token_id = tf.random.categorical(tf.expand_dims(preds, 0), num_samples=1).numpy()[0, 0]
        else:
            # Si temperature est 0 ou moins, on revient à argmax (choix déterministe)
            sampled_token_id = np.argmax(preds)

        # 6. Convertir l'ID en mot
        sampled_word = target_id_to_word.get(sampled_token_id, "[UNK]")
        print(f"  Iteration {i}: Sampled Token ID = {sampled_token_id}, Sampled Word = {sampled_word}")
        i+=1
        # 7. Condition d'arrêt
        if sampled_word == "[end]" or i > max_target_seq_len:
            stop_condition = True
        else:
            decoded_sentence += sampled_word + " "

        # 8. Mettre à jour la séquence cible (pour le prochain tour)
        target_seq[0, 0] = sampled_token_id

        # 9. Mettre à jour les états
        states_value = [h, c]

    return decoded_sentence.strip()

# --- Tester la traduction ---
print("\n===== Test de Traduction =====")

for i in [0]:
    input_text = input_texts[i]
    target_text = target_texts[i].replace("[start]", "").replace("[end]", "").strip()
    translation = translate(input_text, temperature=0.0)

    print("\n---------------------")
    print(f"Anglais : {input_text}")
    print(f"Français (vrai) : {target_text}")
    print(f"Français (prédit) (temp=0.0): {translation}")

# Test with a higher temperature (e.g., 0.7)
for i in [0]: # Testing with 'Run!'
    input_text = input_texts[i]
    target_text = target_texts[i].replace("[start]", "").replace("[end]", "").strip()
    translation = translate(input_text, temperature=0.01)

    print("\n---------------------")
    print(f"Anglais : {input_text}")
    print(f"Français (vrai) : {target_text}")
    print(f"Français (prédit) (temp=0.7): {translation}")


===== Test de Traduction =====

Translating: 'Go.' (temperature=0.0)
  Iteration 0: Sampled Token ID = 1, Sampled Word = [UNK]
  Iteration 1: Sampled Token ID = 1, Sampled Word = [UNK]
  Iteration 2: Sampled Token ID = 3, Sampled Word = [end]

---------------------
Anglais : Go.
Français (vrai) : Va !
Français (prédit) (temp=0.0): [UNK] [UNK]

Translating: 'Go.' (temperature=0.01)
  Iteration 0: Sampled Token ID = 0, Sampled Word = 


/tmp/ipython-input-3860372460.py:42: RuntimeWarning: invalid value encountered in divide
  preds = exp_preds / np.sum(exp_preds) # Re-normaliser


  Iteration 1: Sampled Token ID = 215, Sampled Word = trois
  Iteration 2: Sampled Token ID = 22, Sampled Word = suis
  Iteration 3: Sampled Token ID = 0, Sampled Word = 
  Iteration 4: Sampled Token ID = 0, Sampled Word = 
  Iteration 5: Sampled Token ID = 77, Sampled Word = besoin
  Iteration 6: Sampled Token ID = 168, Sampled Word = ces
  Iteration 7: Sampled Token ID = 0, Sampled Word = 
  Iteration 8: Sampled Token ID = 188, Sampled Word = simplement
  Iteration 9: Sampled Token ID = 132, Sampled Word = voiture
  Iteration 10: Sampled Token ID = 135, Sampled Word = estu
  Iteration 11: Sampled Token ID = 58, Sampled Word = comment
  Iteration 12: Sampled Token ID = 227, Sampled Word = marie
  Iteration 13: Sampled Token ID = 100, Sampled Word = rien
  Iteration 14: Sampled Token ID = 248, Sampled Word = aider
  Iteration 15: Sampled Token ID = 118, Sampled Word = voir
  Iteration 16: Sampled Token ID = 240, Sampled Word = l'école
  Iteration 17: Sampled Token ID = 21, Sampled Word